In [1]:
#include "../common.hpp"

# Efficiency & Expressiveness
## Recap
### Definition of _Type_ and _Regular_
### Semantics and Complexity
### Computational Basis and Computationally Complete
### Equationally Complete
### Whole/Part Relationship
### Safety
### Canonical Type with and without Remote Parts

In [2]:
namespace bcc {

class simple_type {
    int _members = 0;

public:
    simple_type() noexcept = default;                         // default-ctor

    simple_type(const simple_type&) = default;                // copy-ctor
    simple_type& operator=(const simple_type&) = default;     // copy-assign

    simple_type(simple_type&&) noexcept = default;            // move-ctor
    simple_type& operator=(simple_type&&) noexcept = default; // move_assign

    friend bool operator==(const simple_type& a, const simple_type& b) {
        return tie(a._members /*, ...*/) == tie(b._members /*, ...*/);
    }
    friend bool operator!=(const simple_type& a, const simple_type& b) {
        return !(a == b);
    }
};

} // namespace bcc

In [3]:
namespace bcc {

class pimpl_type {
    class implementation;
    struct deleter {
        void operator()(implementation*) const;
    };
    unique_ptr<implementation, deleter> _remote;

public:
    pimpl_type() noexcept = default;                        // default-ctor
    pimpl_type(const pimpl_type&);                          // copy-ctor
    pimpl_type& operator=(const pimpl_type& a) {            // copy-assign
        return *this = pimpl_type(a);
    }
    pimpl_type(pimpl_type&&) noexcept = default;            // move-ctor
    pimpl_type& operator=(pimpl_type&&) noexcept = default; // move_assign
    friend bool operator==(const pimpl_type&, const pimpl_type&);
    friend bool operator!=(const pimpl_type& a, const pimpl_type& b) {
        return !(a == b);
    }
};

} // namespace bcc

In [4]:
// cpp file
namespace bcc {

struct pimpl_type::implementation {
    // a simple type...
    int _members = 0;

    friend bool operator==(const implementation& a, const implementation& b) {
        return tie(a._members /*, ...*/) == tie(b._members /*, ...*/);
    }
};

void pimpl_type::deleter::operator()(implementation* a) const { delete a; }

pimpl_type::pimpl_type(const pimpl_type& a)
    : _remote(a._remote ? new implementation(*a._remote) : nullptr) {}

bool operator==(const pimpl_type& a, const pimpl_type& b) {
    return *a._remote == *b._remote;
}

} // namespace bcc

- In both cases the default-dtor is used (not specified)
- We will be covering polymorphic types and containers later in the course

## Prior Homework

**Exercise:** Look at the regular operations (copy, assignment, equality, default construction) for ZString (or a commonly used type within your project). Is the implementation correct? Complete? Efficient?

`ZString` operations:
- default-ctor: Should be declared `noexcept` but will not throw
- copy-ctor: Logical copy by incrementing reference count to immutable string, should be declared `noexcept`.
- copy-assign: Handles self assignment, requires locking (spin-lock). Complex logic. Benchmark against a copy/move implementation? Returns void?
- move-ctor: Should be declared `noexcept` but will not throw, expensive operation to atomic increment a reference count on `TheOneTrueEmptyZByteRun`, guarantees moved from `ZString` is empty string.
- move-assign: Implemented as swap(). Does not guarantee moved from `ZString` is empty.
- equality: Representational (not value) equality.

Observation: `fDefaultRun` is hardly used except for test cases and to propagate `fCharacterRun`. Is it needed?

Discussion: How can we incrementally improve ZString?

## What Should Be Part of The Public Interface On A Type?

- In general we want the minimum number of public calls (basis operations) to provide a type which is:
    - Computationally Complete
    - Equationally Complete
    
- Other operations should be implemented in terms of those basis operations

Example:

In [5]:
namespace bcc {

class natural_number {
    unsigned int _data = 0;

public:
    // default standard operations
    natural_number& operator++() {
        ++_data;
        return *this;
    }
    natural_number& operator--() {
        --_data;
        return *this;
    }
    bool is_zero() const { return _data == 0; }
};

} // namespace bcc

- `natural_number` is computationally and equationally complete

In [6]:
namespace bcc {

bool operator==(natural_number a, natural_number b) {
    while (!a.is_zero() && !b.is_zero()) {
        --a;
        --b;
    }
    return a.is_zero() && b.is_zero();
}

} // namespace bcc

- Being correct and complete is not enough

## Efficient Basis

- An operation is _efficient_ if there is no way to implement it to use fewer resources
    - time
    - space
    - energy
    
- Unless otherwise specified, we will use efficiency to mean _time efficiency_
    - But in practice, where not all three can be achieved the trade-offs should be considered

- A type has an _efficient basis_ if any additional operations can be implemented efficiently in terms of the basis operations. Making all data members public ensures an efficient basis, but may be unsafe
- In fact, we can prove that some operations cannot be implemented both efficiently and safely
- The canonical example is in-situ sort, although it is true of any in-situ permutation
    - This is why functional languages do not allow direct in-situ permutations

- In C++, move is both unsafe and inefficient
    - It is less safe than copy
    - But more efficient than copy
    
- When writing a type,

## Expressive Basis